In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import DataRetriever as dr

RETRIEVER = dr.DataRetriever()
CON_ATTRIBUTES = RETRIEVER.get_attributes(file_name='consuming_attributes.pkl')
PV_ATTRIBUTES = RETRIEVER.get_attributes(file_name='producing_attributes.pkl')
FLEX_ATTRIBUTES = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
FIXED_ATTRIBUTES = list(set(CON_ATTRIBUTES) - set(FLEX_ATTRIBUTES))

PV = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
FLEX = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FLEX_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
FIXED = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

In [3]:
PV

Timestamp
2015-02-01 00:00:00    0.018496
2015-02-01 01:00:00    0.018332
2015-02-01 02:00:00    0.018385
2015-02-01 03:00:00    0.018502
2015-02-01 04:00:00    0.018524
                         ...   
2016-01-31 19:00:00    0.017710
2016-01-31 20:00:00    0.017940
2016-01-31 21:00:00    0.018149
2016-01-31 22:00:00    0.018209
2016-01-31 23:00:00    0.018283
Freq: H, Length: 8760, dtype: float64

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=PV.index,
                         y=PV,
                         name='PV',
                         mode='lines',
                         line=dict(color='royalblue', width=3)))

fig.add_trace(go.Scatter(x=FIXED.index,
                         y=FIXED,
                         name='Fixed',
                         mode='lines',
                         line=dict(color='firebrick', width=3)))

fig.add_trace(go.Scatter(x=FLEX.index,
                         y=FLEX,
                         name='Flexible',
                         mode='lines',
                         line=dict(color='firebrick', width=3, dash='dot')))

fig.add_trace(go.Scatter(x=PV.index,
                         y=np.subtract(PV, np.add(FIXED, FLEX)),
                         name='Balance',
                         mode='lines',
                         line=dict(color='rgba(0, 0, 0, 0.75)', width=4),
                         fill='tozeroy',
                         fillcolor="rgba(80, 80, 80, 0.25)"))

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                  legend=dict(orientation="h",
                              yanchor="top",
                              y=1.06,
                              xanchor="left",
                              x=0.0))

#x axis
# fig.update_xaxes(visible=False)

#y axis
# fig.update_yaxes(visible=False)

fig.update_xaxes(showline=True, linewidth=2, linecolor='grey', title="")
fig.update_yaxes(showline=True, linewidth=2, linecolor='grey', title="kWh")

fig.show()

In [5]:
df = pd.concat([PV, FLEX, FIXED], axis=1)
df.rename(columns={0:"PV", 1:"FLEX", 2:"FIXED"}, inplace=True)
df["Balance"] = df["PV"] - (df["FLEX"] + df["FIXED"])
df

,PV,FLEX,FIXED,Balance
Timestamp,,,,
2015-02-01 00:00:00,0.018496,0.002947,1.748570,-1.733021
2015-02-01 01:00:00,0.018332,0.002947,2.216490,-2.201106
2015-02-01 02:00:00,0.018385,0.002947,1.941349,-1.925911
2015-02-01 03:00:00,0.018502,0.002947,1.750880,-1.735325
2015-02-01 04:00:00,0.018524,0.002947,1.979749,-1.964172
...,...,...,...,...
2016-01-31 19:00:00,0.017710,0.002947,1.016179,-1.001416
2016-01-31 20:00:00,0.017940,0.010082,0.647665,-0.639806
2016-01-31 21:00:00,0.018149,0.376979,0.962249,-1.321079


In [6]:
df["FLEX"][df["Balance"] < 0].sum()

340.73042788712905

In [7]:
df["FLEX"].sum()

633.513443883432

In [8]:
df["FIXED"].sum()

11627.314411535915

In [9]:
df["FLEX"].sum() / (df["FIXED"].sum() + df["FLEX"].sum())

0.051669711976538024